In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append('../Customized_Components') #this path is not right for this PC. Maybne this will work?
from bandaged_dolan import DolanJunctionBandage
from houcklab_qubit import DiffTransmonRounded

import warnings

import shapely
from CoupledLineTee import CoupledLineTee
from dolan_junction import DolanJunction
from LaunchpadWirebondCustom import LaunchpadWirebondCustom
from shapely.errors import ShapelyDeprecationWarning
from single_pad_transmon_pocket import TransmonPocket_Single

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
from collections import OrderedDict

import Default_res_params as dp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy

# from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
# import pyEPR as epr
from qiskit_metal import Dict, Headings, MetalGUI, designs, draw
# from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
# from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
# from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
# from qiskit_metal.qlibrary.tlines.mixed_path import RouteMeander, RouteMixed
# from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
# from qiskit_metal.toolbox_metal.parsing import *


In [2]:
gui = MetalGUI(dp.design)
design = dp.design
design.overwrite_enabled = True

## Calculate Qubit Params

In [3]:
import Transmon_property as tp

In [4]:
import scqubits as scq
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c

from scipy.optimize import root_scalar

In [5]:
ratio = 65
Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

def inductance_to_cap(L,ratio = ratio):
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    return (c.e.si**2 / (2 * EC)/c.h).to(u.fF)

def target_function(L, desired_frequency, ratio):
    L = L * u.nH  # Convert L to nanohenrys
    calculated_frequency = calculate_qubit_freq(L, ratio)
    return calculated_frequency - desired_frequency
def calculate_qubit_freq(L,ratio = ratio):
    
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    ng = 0.0   # Offset charge
    nlev = 10  # Number of energy levels to consider

    # Create a Transmon object
    transmon = scq.Transmon(EJ=EJ.value, EC=EC.value, ng=ng, ncut=30, truncated_dim=nlev)

    # Calculate the eigenenergies
    eigenenergies = transmon.eigenvals()

    # Calculate the transition frequencies (in GHz)
    transition_frequencies = np.diff(eigenenergies)

    # Calculate the anharmonicity (in GHz)
    anharmonicity = transition_frequencies[1] - transition_frequencies[0]
    return transition_frequencies[0]
    
class TransmonQubit():
    def __init__(self, freq, ratio):
        self.freq = freq
        self.ratio = ratio
        self.C = None,
        self.L = None
        self.EJ = None
        self.EC = None
        self.update_params()
        self.alpha = self.EC
        
    def update_params(self):
        self.C, self.L = self.qubit_freq_to_cap()
        Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

        # Calculate the Josephson energy EJ and charging energy EC
        EJ = (((Phi0/2/np.pi)**2/self.L).to(u.J)/c.h).to(u.GHz)#, equivalencies=u.spectral())  # Convert to GHz
        EC = (c.e.si**2 / (2 * self.C)/c.h).to(u.GHz)
        self.EJ = EJ
        self.EC = EC
        
    def get_params(self):
        return [self.freq, self.ratio, self.qubit_capacitance, self.qubit_inductance, self.EJ, self.EC]
    def get_qcap(self):
        return self.C
    
    def get_qind(self):
        return self.L
    
    def get_EJ(self):
        return self.EJ
    
    def get_EC(self):
        return self.EC
    
    def qubit_freq_to_cap(self, ):
        desired_frequency = self.freq   # Desired frequency in GHz
        ratio = self.ratio
        # Use scipy.optimize.root_scalar to find the root
        def target_function(L, desired_frequency,):
            self.L = L * u.nH  # Convert L to nanohenrys
            self.calculate_qubit_freq()
            calculated_frequency = self.freq.to(u.GHz).value
            return calculated_frequency - desired_frequency
        result = root_scalar(target_function, args=(desired_frequency.to(u.GHz).value,), bracket=[1, 100], method='brentq')

        # Print the result
        if result.converged:
            L_optimal = result.root
            print(f"Optimal inductance L: {L_optimal} nH")
        else:
            print("Root finding did not converge")
        c_optimal = inductance_to_cap(L_optimal*u.nH, ratio = ratio)
        return c_optimal, L_optimal*u.nH
    def calculate_qubit_freq(self,):
        L = self.L
        ratio = self.ratio
        EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
        #, equivalencies=u.spectral())  # Convert to GHz
        EC = EJ/ratio
        ng = 0.0   # Offset charge
        nlev = 10  # Number of energy levels to consider

        # Create a Transmon object
        transmon = scq.Transmon(EJ=EJ.value, EC=EC.value, ng=ng, ncut=30, truncated_dim=nlev)

        # Calculate the eigenenergies
        eigenenergies = transmon.eigenvals()

        # Calculate the transition frequencies (in GHz)
        transition_frequencies = np.diff(eigenenergies)

        # Calculate the anharmonicity (in GHz)
        anharmonicity = transition_frequencies[1] - transition_frequencies[0]
        self.freq = transition_frequencies[0]*u.GHz
        self.alpha = anharmonicity*u.GHz
    

In [6]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)

Optimal inductance L: 13.676453476080171 nH


In [7]:
qubit.EC

<Quantity 0.18387753 GHz>

In [8]:
qubit.alpha

<Quantity 0.18387753 GHz>

Design specifics

In [9]:
from Transmon_specifications import find_junction_area, find_junction_capacitance

In [10]:
def pad_len_from_C(c):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*c.to(u.fF).value + intercept 
def unitless_find_j_area(L):
    return find_junction_area(L*u.nH,Jc = 0.42*u.uA/u.um**2).to(u.um**2).value

def unitless_find_pad_len(C):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*C + intercept

In [11]:
def convert_capacitance_to_energy(capacitance):
    C = capacitance*u.fF
    return (c.e.si**2 / (2 * C)/c.h).to(u.GHz)

def get_cap_params(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,0,-1,0],
              [0,1,0,0],
              [1,0,1,0],
              [0,0,0,1]])
    cap_matrix[0,2] -= cj
    cap_matrix[2,0] -= cj
    cap_matrix[2,2] += cj
    cap_matrix[0,0] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[0,0]
    coupling_capacitance = np.reciprocal(inverse)[0,1]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    g = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    return [Ec,g,qubit_capacitance,coupling_capacitance]
    

In [12]:
get_cap_params(np.array([[80,0,0,0],[0,80,0,0],[0,0,80,0],[0,0,0,80]]),0.5)

[<Quantity 0.48425573 GHz>, <Quantity 0. MHz>, 40.0, inf]

## Resonator calculations

In [13]:
import astropy.units as u
import astropy.constants as c
import scipy.optimize as optimize

# define constants for LL designs
sub_t = 350*u.um #substrate thickness (Si in this case)
metal_t = 100*u.nm #Deposited metal thickness (Al)
Sc = 67*u.fF/(u.um)**2 #JJ specific capacitance
epsilon = 11.45
W_jj = 200*u.nm #junction width
Z0 = 50*u.Ohm #characteristic impedance
import qiskit_metal.analyses as analyses

In [14]:
freq = 6*u.GHz.si
line_width = 15*u.um
line_gap = 8.30*u.um

Optimal pin-gap ratio for resonator and feedlines

In [88]:
class ReadoutResonator():
    def __init__(self, freq, short_on_one_end = False,optimal_pin = None, optimal_gap = None ):
        self.freq = freq
        self.short_on_one_end = short_on_one_end
        self.substrate_thickness = 350*u.um
        self.film_thickness = 100*u.nm
        if (optimal_gap is None) and (optimal_pin is None):
            self.pin = 15*u.um
            self.optimize_gap()
        elif optimal_gap is None:
            self.pin = optimal_pin
            self.optimize_gap()
        elif optimal_pin is None:
            self.gap = optimal_gap
            self.optimize_pin()
        
        self.param_from_res_freq(freq, short_on_one_end = short_on_one_end)
        self.C = np.pi/2/self.freq/2/np.pi/self.Z0
        
    def get_params(self):
        return [self.L, self.C, self.target_length]
    
    def get_L(self):
        return self.L
    
    def get_C(self):
        return self.C
    
    def get_target_length(self):
        return self.target_length
    
    def param_from_res_freq(self,freq, short_on_one_end = False):
        self.Lk, self.Lext,self.Cl,self.G,self.Z0,self.etf,self.Cstar = analyses.cpw_calculations.lumped_cpw(freq = self.freq.to(u.Hz).value,
                                        line_width=self.pin.to(u.m).value,
                                        line_gap=self.gap.to(u.m).value,
                                        substrate_thickness=self.substrate_thickness.to(u.m).value,
                                        film_thickness=self.film_thickness.to(u.m).value,)
        self.Lk*=u.H
        self.Lext*=u.H
        self.Cl*=u.F/u.m
        
        self.G*=u.S
        self.Z0*=u.Ohm
        self.etf*=u.H/u.m
        self.Cstar*=u.F
        target_length = analyses.cpw_calculations.guided_wavelength(freq = self.freq.to(u.Hz).value,
                                        line_width=self.pin.to(u.m).value,
                                        line_gap=self.gap.to(u.m).value,
                                        substrate_thickness=self.substrate_thickness.to(u.m).value,
                                        film_thickness=self.film_thickness.to(u.m).value,)[0]*u.m
        if short_on_one_end:
            target_length /= 4
        else:
            target_length /= 2
        self.len = target_length
    def optimize_gap(self):
        def f(line_gap):
            return(analyses.cpw_calculations.lumped_cpw(self.freq.to(u.Hz).value,
                                            self.pin.to(u.m).value,
                                            line_gap,
                                            self.substrate_thickness.to(u.m).value,
                                            self.film_thickness.to(u.m).value)[4]-50)
        optimized_gap = optimize.fsolve(f,self.pin.si.value)[0]*u.m
        self.gap = optimized_gap.to(u.um)
    def optimize_pin(self):
        def f(line_width):
            return(analyses.cpw_calculations.lumped_cpw(self.freq.to(u.Hz).value,
                                            line_width,
                                            self.gap.si.value,
                                            self.substrate_thickness.to(u.m).value,
                                            self.film_thickness.to(u.m).value)[4]-50)
        optimized_pin = optimize.fsolve(f,self.gap.si.value)[0]*u.m
        self.pin = optimized_pin.to(u.um)

In [89]:
res = ReadoutResonator(6*u.GHz, short_on_one_end = False, optimal_pin = 20*u.um)

In [90]:
class DispersiveReadout():
    def __init__(self, qubit, res, T1, c_qr = None, c_tr = None):
        self.qubit = qubit
        self.res = res
        self.pT1 = T1
        self.optimize_cqr()
        self.optimize_ctr()
        
    def get_params(self):
        return [self.qubit, self.res, self.c_qr, self.c_tr]
    
    def get_cqr(self):
        return self.c_qr
    
    def get_ctr(self):
        return self.c_tr
    
    def get_qubit(self):
        return self.qubit
    
    def get_res(self):
        return self.res
    def calc_g(self):
        c_qr = self.c_qr
        Cq = self.qubit.C
        Cr = self.res.C
        wq = self.qubit.freq*np.pi*2
        wr = self.res.freq*np.pi*2
        EJ = self.qubit.EJ
        EC = self.qubit.EC
        return c_qr/Cq*np.sqrt(c.e.si**2*wr/c.hbar/Cr)*(EJ/8/EC)**0.25
    def calc_kappa(self):
        res = self.res
        c_tr = self.c_tr
        return (analyses.em.kappa_calculation.kappa_in(res.freq.si.value,
                                       c_tr.si.value,
                                       res.freq.si.value)*u.Hz).to(u.MHz)
    def calc_chi(self):
        qubit = self.qubit
        resonator = self.res
        g_val = self.calc_g()
        delta = (qubit.freq-resonator.freq)*2*np.pi
        sum_freq = (qubit.freq+resonator.freq)*2*np.pi
        alpha = qubit.EC
        return 2*g_val**2*(alpha/delta/(delta+alpha)+alpha/sum_freq/(sum_freq+alpha))
    def calc_purcellT1(self,):
        qubit = self.qubit
        res = self.res
        kappa_val = self.calc_kappa()
        g_val = self.calc_g()
        delta = (qubit.freq-res.freq)*2*np.pi
        
        purcellGamma = g_val**2*kappa_val/delta**2
        
        return 1/purcellGamma/2/np.pi
    def calc_purcellT1q(self,):
        '''taking kappa = 2*chi for the optimal SNR, can directly get the T1q'''
        c_qr = self.c_qr
        qubit = self.qubit
        res = self.res
        g_val = self.calc_g()
        delta = (qubit.freq-res.freq)*2*np.pi
        
        chi_val = self.calc_chi()
        kappa_val = chi_val*2
        
        purcellGamma = g_val**2*kappa_val/delta**2
        
        return 1/purcellGamma/2/np.pi
    def optimize_cqr(self,):
        '''find the optimal coupling capacitance'''
        target_t1 = self.pT1
        c_qr = 1*u.fF
        self.c_qr = c_qr
        while self.calc_purcellT1q().to(u.ms) > 1.01*target_t1:
            c_qr += .1*u.fF
            self.c_qr = c_qr
        return c_qr
    def optimize_ctr(self, target_kappa=None):
        '''find the optimal coupling capacitance'''
        if target_kappa is None:
            target_kappa = self.calc_chi()*2
        c_tr = 1*u.fF
        self.c_tr = c_tr
        while self.calc_kappa() < target_kappa:
            c_tr += .1*u.fF
            self.c_tr = c_tr
        # return c_tr


In [91]:
res.Z0

<Quantity 50. Ohm>

In [92]:
res.C

<Quantity 0.00083333 1 / (GHz Ohm)>

In [93]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)
dr = DispersiveReadout(qubit,res,5*u.ms)

Optimal inductance L: 13.676453476080171 nH


In [19]:
c1, renderer_q3d, q3d = dp.init_q3d_sim()

In [20]:
default_options = Dict(
        cut_l='800um',
        cut_h='1200um',
        gap='20um',
        w = '90um',
        l = '900um',
        r = '60um',
        cpw_l = '50um',
        coupling_gap = '40um',
        JJ_cutout_w = '70um',
        JJ_cutout_h = '70um',
        JJ_cutout_r = '150um',
        JJ_c_contact_l = '40um',
        JJ_c_contact_r = '2.5um',
        JJ_c_contact_w = '10um',
        coupling_d = '100um',
        coupling_pad_w = '90um',
        coupling_r = '40um',
        cpw_pin = '10um',
        chip='main',
        resolution = '10',
        junction = 'False',
        orientation = '0',
        JJ_c_contact_shortl = '10um',
        istunnel = 'False')

In [21]:
raise Exception('stop!')

Exception: stop!

## Qiskit Metal

In [78]:
qubit.L.to(u.nH)

<Quantity 13.67645348 nH>

In [23]:
default_options['coupling_d'] = '100um'
default_options['coupling_gap'] = '100um'
default_options['gap'] = '50um'
default_options['l'] = '900um'

for i,l in enumerate([1000]):
    default_options['l'] = f'{l}um'
    q1 = DiffTransmonRounded(design,'Q1',options = default_options)
    gui.rebuild()
    
    c1.sim.run(components = ['Q1'],open_terminations = [('Q1','cpw_stub')])
    print(l)
    print(c1.sim.capacitance_matrix)
    
    
    c1.setup.junctions = Dict({'Lj': qubit.L.to(u.nH).value, 'Cj': 2})
    c1.setup.freq_readout = 6.0
    c1.setup.freq_bus = []

    result = c1.run_lom()
    # print(c1.lumped_oscillator)
    last_row = result.iloc[-1:]
    for keys in default_options.keys():
        last_row.loc[len(result),keys] = design.parse_value(default_options[keys])
    
    
    if i ==0:
        df = last_row
    else:
        df = df.append(last_row, ignore_index=True,)

    df.to_csv('pad_length_sweep.csv')
    
# q1 = DiffTransmonRounded(design,'Q1',options = default_options)
# gui.rebuild()

INFO 06:52PM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:52PM [load_ansys_project]: 	Opened Ansys App
INFO 06:52PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 06:52PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/slab/Documents/Ansoft/
	Project:   Project48
INFO 06:52PM [connect_design]: No active design found (or error getting active design).
INFO 06:52PM [connect]: 	 Connected to project "Project48". No design detected
INFO 06:52PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 06:52PM [connect_setup]: 	No design setup detected.
WARNING 06:52PM [connect_setup]: 	Creating Q3D default setup.
INFO 06:52PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 06:52PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 06:52PM [analyze]: Analyzing setup Setup
INFO 06:53PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Tem

1000
                   ground_main_plane  pad_left_Q1  pad_right_Q1  \
ground_main_plane          284.38954   -103.35637     -75.22100   
pad_left_Q1               -103.35637    184.90850     -67.99716   
pad_right_Q1               -75.22100    -67.99716     147.17341   
resonator_pad_Q1           -15.95247    -10.45159      -1.40574   

                   resonator_pad_Q1  
ground_main_plane         -15.95247  
pad_left_Q1               -10.45159  
pad_right_Q1               -1.40574  
resonator_pad_Q1           28.08333  


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 C:\Users\slab\AppData\Local\Temp\ipykernel_20728\1954483913.py: 24
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 C:\Users\slab\AppData\Local\Temp\ipykernel_20728\1954483913.py: 24
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 C:\Users\slab\AppData\Local\Temp\ipykernel_20728\1954483913.py: 24
A value

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 3.824216 [GHz]
EC 167.286153 [MHz]
EJ 11.947220 [GHz]
alpha -186.961134 [MHz]
dispersion 0.213693 [KHz]
Lq 13.670938 [nH]
Cq 115.790983 [fF]
T1 2859.854395 [us]

**Coupling Properties**

tCqbus1 3.365389 [fF]
gbus1_in_MHz 26.915581 [MHz]
χ_bus1 -0.055555 [MHz]
1/T1bus1 55.651415 [Hz]
T1bus1 2859.854395 [us]
Bus-Bus Couplings


In [ ]:
try:
    df.append(last_row, ignore_index=True)
except:
    print('err')

 C:\Users\slab\AppData\Local\Temp\ipykernel_24816\2998703508.py: 2


In [ ]:
df['l']

10    0.8
Name: l, dtype: float64

In [ ]:
(result['fQ'].values).astype(float)

array([4.00746608, 3.98946818, 3.92705587, 3.88067617, 3.83296908,
       3.80953637, 3.78456473, 3.77685466, 3.75988887, 3.75206023,
       3.7509349 ])

In [ ]:
last_row = result.iloc[-1:]

In [ ]:
last_row

,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz,cut_l,...,coupling_d,coupling_pad_w,coupling_r,cpw_pin,chip,resolution,junction,orientation,JJ_c_contact_shortl,istunnel
11,3.750935,153.274204,12.4689,-169.884754,0.046577,[-48.52351835942086],[-0.1556113946580644],0.155611,48.523518,0.8,...,0.1,0.09,0.04,0.01,main,10.0,False,0.0,0.01,False


In [ ]:
for keys in default_options.keys():
    last_row.loc[11,keys] = design.parse_value(default_options[keys])

In [ ]:
last_row

,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz,cut_l,...,coupling_d,coupling_pad_w,coupling_r,cpw_pin,chip,resolution,junction,orientation,JJ_c_contact_shortl,istunnel
11,3.750935,153.274204,12.4689,-169.884754,0.046577,[-48.52351835942086],[-0.1556113946580644],0.155611,48.523518,0.8,...,0.1,0.09,0.04,0.01,main,10.0,False,0.0,0.01,False


In [ ]:
c1.sim.run(components = ['Q1'],open_terminations = [('Q1','cpw_stub')])

INFO 06:24PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 06:24PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 06:24PM [analyze]: Analyzing setup Setup
INFO 06:25PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp4tvojshh.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 06:25PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpm_o9nmak.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 06:25PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp4c2euq_s.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 06:25PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp0czjex61.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, 

## Leonding method

In [53]:
def convert_capacitance_to_energy(capacitance):
    C = capacitance*u.fF
    return (c.e.si**2 / (2 * C)/c.h).to(u.GHz)

def get_cap_params(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,0,0,0],
                  [0,1,1,0],
                  [0,1,-1,0],
                  [0,0,0,1]])
    cap_matrix[1,2] -= cj
    cap_matrix[2,1] -= cj
    cap_matrix[2,2] += cj
    cap_matrix[1,1] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[2,2]
    coupling_capacitance = np.reciprocal(inverse)[2,3]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    J = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    return [Ec,J,qubit_capacitance,coupling_capacitance]

In [56]:
res.C = (np.pi/2/res.freq/2/np.pi/res.Z0).to(u.fF)

In [58]:
res.C

<Quantity 833.33333333 fF>

In [ ]:
df = c1.sim.capacitance_matrix


In [79]:
df

,ground_main_plane,pad_left_Q1,pad_right_Q1,resonator_pad_Q1
ground_main_plane,1117.722873,-103.35637,-75.22100,-849.285803
pad_left_Q1,-103.356370,184.90850,-67.99716,-10.451590
pad_right_Q1,-75.221000,-67.99716,147.17341,-1.405740
resonator_pad_Q1,-849.285803,-10.45159,-1.40574,861.416663


In [55]:
df1 = df

In [59]:
df1['ground_main_plane']['ground_main_plane'] += res.C.to(u.fF).value
df1['resonator_pad_Q1']['resonator_pad_Q1'] += res.C.to(u.fF).value
df1['resonator_pad_Q1']['ground_main_plane'] -= res.C.to(u.fF).value
df1['ground_main_plane']['resonator_pad_Q1'] -= res.C.to(u.fF).value


In [60]:
df1

,ground_main_plane,pad_left_Q1,pad_right_Q1,resonator_pad_Q1
ground_main_plane,1117.722873,-103.35637,-75.22100,-849.285803
pad_left_Q1,-103.356370,184.90850,-67.99716,-10.451590
pad_right_Q1,-75.221000,-67.99716,147.17341,-1.405740
resonator_pad_Q1,-849.285803,-10.45159,-1.40574,861.416663


In [67]:
cap = df1.to_numpy()
Ec, J,_,_ = get_cap_params(cap,2)

In [62]:
def qubit_zpf(E_J, E_c):
    z_Q = (1/2*np.pi)*np.sqrt(8*E_c/E_J)
    return np.sqrt(1/(4*np.pi*z_Q))
def res_zpf(Z_0 = 50*u.Ohm):
    R_Q = c.h/(2*c.e.si**2)
    Z = 2*Z_0/np.pi
    z= Z/R_Q
    return np.sqrt(1/(4*np.pi*z))
    
    

In [68]:
phi0 = c.h/2/c.e.si
T = 30*u.mK
epsilon_r = 11.45


Cq = 20*u.fF
Lj = 13*u.nH
# Ec = (c.e.si**2/2/Cq)/c.h
Ej = ((phi0/2/np.pi)**2/Lj)/c.h

In [69]:
Ec

<Quantity 0.16528966 GHz>

In [70]:
qubit_zpf(Ej, Ec)

<Quantity 0.39524761>

In [71]:
J

<Quantity 8.85184138 MHz>

In [73]:
(qubit_zpf(Ej,Ec)*J*res_zpf()).to(u.MHz)

<Quantity 19.87355232 MHz>

In [74]:
Ec

<Quantity 0.16528966 GHz>

In [ ]:
res.len

<Quantity 0.01006103 m>

In [ ]:
qubit = TransmonQubit(3.9*u.GHz, ratio = 75)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)
dr = DispersiveReadout(qubit,res,5*u.ms)

Optimal inductance L: 13.104251477852955 nH


In [ ]:
qubit.alpha = 0.2*u.GHz
dr.calc_g().to(u.MHz)

<Quantity 192.11783453 MHz>

In [ ]:
c1.setup.junctions = Dict({'Lj': qubit.L.to(u.nH).value, 'Cj': 2})
c1.setup.freq_readout = 6.0
c1.setup.freq_bus = []

c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 3.690060 [GHz]
EC 148.114964 [MHz]
EJ 12.468900 [GHz]
alpha -163.828683 [MHz]
dispersion 0.030310 [KHz]
Lq 13.098967 [nH]
Cq 130.778333 [fF]
T1 600.611494 [us]

**Coupling Properties**

tCqbus1 -8.692653 [fF]
gbus1_in_MHz -63.738432 [MHz]
χ_bus1 -0.247370 [MHz]
1/T1bus1 264.988174 [Hz]
T1bus1 600.611494 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,3.937209,169.666964,12.4689,-189.263737,0.159523,[-68.74971808495071],[-0.40359764269585285],0.403598,68.749718
2,3.915711,167.72814,12.4689,-186.960931,0.139235,[-63.63001386776019],[-0.33549616805997096],0.335496,63.630014
3,3.832352,160.326385,12.4689,-178.196203,0.080971,[-63.94164978919576],[-0.30190565641844364],0.301906,63.941650
4,3.783519,156.075682,12.4689,-173.181869,0.058283,[-63.65935446156925],[-0.2799351524389901],0.279935,63.659354
5,3.759476,154.005819,12.4689,-170.745219,0.049415,[-63.28995398309355],[-0.2678106798699403],0.267811,63.289954
6,3.730663,151.545339,12.4689,-167.853056,0.040432,[-63.17067114264823],[-0.2566120637282528],0.256612,63.170671
7,3.717642,150.440554,12.4689,-166.555968,0.036889,[-63.398800874196056],[-0.2539750505911451],0.253975,63.398801
8,3.703584,149.252767,12.4689,-165.162492,0.033387,[-63.43955528199085],[-0.24954163537966845],0.249542,63.439555
9,3.69276,148.341782,12.4689,-164.094494,0.030903,[-63.54318379491805],[-0.2467486024620791],0.246749,63.543184
10,3.69006,148.114964,12.4689,-163.828683,0.03031,[-63.738432076416636],[-0.2473704176492816],0.247370,63.738432


In [ ]:
c1.lumped_oscillator

{'fQ': 3.6900599513388332,
 'EC': 148.1149642804409,
 'EJ': 12.468899913873381,
 'alpha': -163.82868334418583,
 'dispersion': 0.030310351848602295,
 'gbus': array([-63.73843208]),
 'chi_in_MHz': array([-0.24737042])}

In [ ]:
import copy
cap_matrix = df.to_numpy()
normal = copy.deepcopy(cap_matrix)
cj = 10
T = np.array([[1,0,1,0],
              [0,1,0,0],
              [1,0,-1,0],
              [0,0,0,1]])
cap_matrix[0,2] -= cj
cap_matrix[2,0] -= cj
cap_matrix[2,2] += cj
cap_matrix[0,0] += cj
caps = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
normals = np.linalg.inv(T).T@(normal)@np.linalg.inv(T)

# print(np.reciprocal(np.linalg.inv(normals)))

caps = T@(cap_matrix)@np.linalg.inv(T)
print(np.reciprocal(np.linalg.inv(caps)))
print(caps)
print([ np.reciprocal(np.linalg.inv(caps))[0,0], np.reciprocal(np.linalg.inv(caps))[0,1]])

[[ 3.17472005e+01  3.82385868e+01 -2.72738470e+03  4.91117422e+01]
 [ 7.64771737e+01  2.21929539e+01  2.23275870e+03  9.74883978e+01]
 [-2.72738470e+03  1.11637935e+03  4.19933083e+02  5.85246037e+04]
 [ 9.82234844e+01  9.74883978e+01  1.17049207e+05  9.52129430e+01]]
[[  89.99974   -16.53124    17.53451  -158.36565 ]
 [  -8.26562    30.18888    -6.90227   -13.44846 ]
 [  17.53451   -13.80454   425.30526   -21.20375 ]
 [ -79.182825  -13.44846   -10.601875  261.87651 ]]
[31.747200486251824, 38.23858682521182]


In [ ]:
import copy
cap_matrix = df1.to_numpy()
normal = copy.deepcopy(cap_matrix)
cj = 10
T = np.array([[1/np.sqrt(2),0,1/np.sqrt(2),0],
              [0,1,0,0],
              [1/np.sqrt(2),0,-1/np.sqrt(2),0],
              [0,0,0,1]])
cap_matrix[0,2] -= cj
cap_matrix[2,0] -= cj
cap_matrix[2,2] += cj
cap_matrix[0,0] += cj

cap = T@cap_matrix@T


In [ ]:
print(np.reciprocal(np.linalg.inv(cap)))

[[ 3.14471270e+01  5.39927190e+01 -1.92950701e+03  6.97833844e+01]
 [ 5.39927190e+01  2.21677169e+01  1.19716802e+03  9.79380690e+01]
 [-1.92950701e+03  1.19716802e+03  3.09241760e+02  5.67063089e+04]
 [ 6.97833844e+01  9.79380690e+01  5.67063089e+04  9.56666722e+01]]


## Thesis method

In [26]:
df = c1.sim.capacitance_matrix

In [47]:
phi0 = c.h/2/c.e.si
T = 30*u.mK
epsilon_r = 11.45
z0 = 50*u.Ohm


Cq = 20*u.fF
Lj = 13*u.nH
Ec = (c.e.si**2/2/Cq)/c.h
Ej = ((phi0/2/np.pi)**2/Lj)/c.h

In [75]:
c_1 = -df['pad_left_Q1']['resonator_pad_Q1']*u.fF
c_2 = -df['pad_right_Q1']['ground_main_plane']*u.fF
c_3 = -df['pad_right_Q1']['resonator_pad_Q1']*u.fF
c_4 = -df['pad_left_Q1']['ground_main_plane']*u.fF
ct = -df['pad_left_Q1']['pad_right_Q1']*u.fF

c_phi = (c_2+c_3)*(c_1+c_4)/(c_1+c_2+c_3+c_4)+2*u.fF+ct
c_ceff = (c_1**2-c_3*c_4)/(c_1+c_2+c_3+c_4+ct)
beta = (c_1*c_2-c_3*c_4)/((c_2+c_3)*(c_1+c_4)+ct*(c_1+c_2+c_3+c_4))
wr = 6*u.GHz
Ec = (c.e.si**2/2/c_phi)/c.h
w01 = np.sqrt(8*Ec*Ej)
delta = wr-w01
c_res = np.pi/2/wr/z0
Vrms = np.sqrt(c.hbar*wr/2/c_res)
g = c.e.si*Vrms*beta/c.hbar*np.sqrt(2)*(Ej/8/Ec)**0.25

In [77]:
Ec.to(u.MHz)

<Quantity 167.28616347 MHz>

In [49]:
g.to(u.MHz)

<Quantity 27.34680556 MHz>

In [51]:
Ej.to(u.GHz)

<Quantity 12.57396252 GHz>

In [52]:
w01.to(u.GHz)

<Quantity 4.10214573 GHz>

In [31]:
ct

<Quantity 67.99716 fF>

In [32]:
c_phi

<Quantity 115.79098308 fF>

In [ ]:
T = np.array([[1/np.sqrt(2),0,-1/np.sqrt(2),0],
              [0,1,0,0],
              [1/np.sqrt(2),0,1/np.sqrt(2),0],
              [0,0,0,1]])

In [ ]:
T.T@T

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [ ]:
c1.sim.capacitance_matrix = df

In [ ]:
qubit.L

<Quantity 13.67645348 nH>

In [ ]:
df

,cross_0_Q1,cross_1_Q1,cross_2_Q1,ground_main_plane
cross_0_Q1,275.18701,-15.16789,-167.65276,-89.78470
cross_1_Q1,-15.16789,30.18888,-1.36335,-13.44846
cross_2_Q1,-167.65276,-1.36335,240.11799,-68.58095
ground_main_plane,-89.78470,-13.44846,-68.58095,261.87651


In [ ]:
order = [3,0,2,1]

In [ ]:
import pandas as pd

# Assuming you have a DataFrame called 'df'
# df = pd.DataFrame(...)

# First, swap columns 2 and 0
df2 = df.iloc[:, order]  # Assuming it's a 4x4 matrix

# Then, swap rows 2 and 1
df2 = df2.iloc[order, :]

# If you want to reset the index and column names
df2 = df2.reset_index(drop=True)
df2.columns = range(len(df2.columns))

In [ ]:
df

,cross_0_Q1,cross_1_Q1,cross_2_Q1,ground_main_plane
cross_0_Q1,275.18701,-15.16789,-167.65276,-89.78470
cross_1_Q1,-15.16789,30.18888,-1.36335,-13.44846
cross_2_Q1,-167.65276,-1.36335,240.11799,-68.58095
ground_main_plane,-89.78470,-13.44846,-68.58095,261.87651


In [ ]:
df2

,0,1,2,3
0,261.87651,-89.78470,-68.58095,-13.44846
1,-89.78470,275.18701,-167.65276,-15.16789
2,-68.58095,-167.65276,240.11799,-1.36335
3,-13.44846,-15.16789,-1.36335,30.18888


<Quantity 833.33333333 fF>

In [ ]:
(res.C).to(u.fF)/u.m*res.len

<Quantity 1683.32684491 fF>